In [1]:
import DataAPI as api
import os
import pandas as pd
import numpy as np

start_date = '2019-04-01'
end_date = '2021-06-30'

input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/Original_Input_Shift_1_Sequence_10'
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'
trading_list = api.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % 10
    path_output = os.path.join(output_path, str(flag), str(day.year), day.strftime('%Y%m%d') + '.csv')

    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1